# Text mining 
## HW 2 MaxEnt on Reuters

We are going to train a Multiclass Maximum Entropy (Softmax Regression) to predict the origin of a document coming from the 20newsgroup dataset.

This exercise is similar to 01-LinearRegression. The difference is that you'll have to implement the algorithm yourself.

For this puprose we'll use PyTorch, and sklearn. Your job is to fill in the missing code into the cells below.

You will find the steps you need to perform in the **Task** section in each cell.

In [1]:
%reset -f
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OrdinalEncoder

In [3]:
print('Loading data...')

# Passing none as we want to train over all the data.
newsgroups_train = fetch_20newsgroups(subset='train',
                                      categories=None)

newsgroups_test = fetch_20newsgroups(subset='test',
                                      categories=None)

Loading data...


In [4]:
BATCH_SIZE = 32
MAX_EPOCHS = 20
# Lambda
REG_PARAM = 0.01
# Alpha 
LEARNING_RATE = 1e-02
# Number of features
MAX_WORDS = 10000
# Priting error information after display_step epochs
DISPLAY_STEP = 1
NUM_CLASSES = np.max(newsgroups_train.target) + 1

In [5]:
def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return np.eye(num_classes, dtype='uint8')[y]

In [6]:
print(NUM_CLASSES, 'classes')

print('Vectorizing sequence data...')

vectorizer = TfidfVectorizer(max_features=MAX_WORDS)

tfidf_vectors_train = vectorizer.fit_transform(newsgroups_train.data)
tfidf_vectors_test = vectorizer.transform(newsgroups_test.data)

x_train = tfidf_vectors_train.toarray()
x_test = tfidf_vectors_test.toarray()

y_train = newsgroups_train.target
y_test = newsgroups_test.target

print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)

20 classes
Vectorizing sequence data...
x_train shape: (11314, 10000)
y_train shape: (11314,)
x_test shape: (7532, 10000)
y_test shape: (7532,)


In [7]:
class TwentyNewsGroupsDataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.from_numpy(x)
        self.y = torch.from_numpy(y)

        assert len(self.x) == len(self.y)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

train_dataset = TwentyNewsGroupsDataset(x_train, y_train) 
test_dataset = TwentyNewsGroupsDataset(x_test, y_test)

# Model initilization

Here comes the most interesting part of the model. You'll have to implement Softmax Regression with SGD. The formulas are presented below for you. You don't have to derive them, you can use them as they are, or you can use PyTorch's gradient function to obtain them.

## Softmax regression formulas

*Keep in mind that those are the final formulas, the derivation of gradients has been omitted, but in order to derive them you must use the chain and quotient rules.*

Here is the basic linear (activation) function:

$ z_i = x^T w_i + b_i$

This is the softmax (prediction) for class i:

$\hat{y}_i = \sigma(\textbf{z})_i = \frac{\exp(z_i)}{\sum_{k=1}^{K}{\exp(z_k)}}$

Derivative of the softmax wrt the activation, here $1(i = j)$ is the identity function, which is $1$ if $i = j$ and $0$ otherwise:

${\frac{\partial}{\partial w_j} \sigma(\textbf{z})_i = \sigma(\textbf{z})_i}(1(i = j) - {\sigma(\textbf{z})_j})\ x$

Negative cross-entropy, note that this is a dot product of $y$ and $\hat{y}$, which are K dimentional vectors (y is K dimentional vector with 1 in correct class and 0 everywhere else, so it can be omitted for other classes).

$\mathcal{L_s} = - \frac{1}{N}\sum_{i = 1}^N y_{i} \log(\hat{y}_{i}) $

Gradient of the loss with respect to the weights (i is the correct class):

$ \frac{\partial }{\partial w_i} \mathcal{L_s} = \hat{y_i}\ x $

Weights update making a step in the direction opposite to the gradient, since we are minimizing the loss and the gradient is always pointing in the direction of the maximim.
Alpha is the learning rate.

$ w_i = w_i - \alpha \frac{\partial }{\partial w_i} \mathcal{L_s} $

Accuracy:

$ Acc(y, \hat{y}) = \frac{1}{N}\sum_{i = 1}^N 1(arg\,max_{j \in K}\ \hat{y}_{i,j} = y_i) $

## Dimentions of components
$ N $ - number of examples

$ M $ - number of features

$ K $ - number of classes

Features input $ x \in {\rm I\!R}^{N \times M} $

Expected class $ y \in {\rm I\!R}^{N \times M} $

Weight matrix $ W \in {\rm I\!R}^{M  \times K} $

Per class bias $ b \in {\rm I\!R}^{K} $

## Tasks
1. Implement softmax regression using the formulas above;
2. Implement accuracy metric, but use cross entropy for optimization. (In the `evaluation` function)

## Tips
Checking the PyTorch's documentation, and the lecture "Introduction to PyTorch". Also you can use all the built-in to compute the gradients!

Also in the loss function you can use the [LogSoftmax](https://pytorch.org/docs/master/nn.html?highlight=log_softmax#torch.nn.LogSoftmax) for numerical stability.

Check the [sub](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.sub_) function of a Tensor, you will most probably need it.


In [10]:
class LogisticRegression(nn.Module):

  def __init__(self, features_size, num_classes):
    super(LogisticRegression, self).__init__()

    self.w = torch.rand((features_size, num_classes))
    self.b = torch.rand(features_size)

  def forward(self, x):
    # logits =  # x + self.b
    y_hat = F.softmax(logits)

    return (logits, y_hat)

model = LogisticRegression(x_train.shape[1], NUM_CLASSES)

In [ ]:
torch.matmul(train_dataset.x.T, model.w)

In [ ]:
dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                        shuffle=True, drop_last=False)
for x, y in dataloader:
    print(x)
    print('-----------')
    print(y)
    cost = update_weights(model, x, y)
    break


In [ ]:
def update_weight(model, x, y):
  # Fit training using batch data and propagate the error
  # Feel free to change as you prefer

  # l2_reg = ...
  logits, _ = model(x)
  loss = F.cross_entropy(logits, y)

  # Update w and b
  

  return loss.detach().cpu()

In [ ]:
def evaluate(model, dataset):
  #fill the evaluation function, you can change parameters if you like
  ...
  pass

# Model training

Train your model with calling the `update_weights` function, and cost computation methods. You don't have to modify this section.

## Sanity check

Your loss should be similar to:

Epoch: 0001 cost=4.237748146  
Epoch: 0002 cost=2.006925821  
Epoch: 0003 cost=0.838360906  
Epoch: 0004 cost=0.526503205  
Epoch: 0005 cost=0.406159312  
Epoch: 0006 cost=0.338935345  
Epoch: 0007 cost=0.288057804  
Epoch: 0008 cost=0.245860726  
Epoch: 0009 cost=0.208140314  
Epoch: 0010 cost=0.170706153  
Epoch: 0011 cost=0.141715422  
Epoch: 0012 cost=0.117129274  
Epoch: 0013 cost=0.094932191  
Epoch: 0014 cost=0.075968713  
Epoch: 0015 cost=0.060179509  
Epoch: 0016 cost=0.049887933  
Epoch: 0017 cost=0.039890103  
Epoch: 0018 cost=0.033839807  
Epoch: 0019 cost=0.027970247  
Epoch: 0020 cost=0.024634583  
Optimization Finished!  

In [ ]:
# Training cycle
def train(model, dataset):
  model.train()
  
  for epoch in range(1, MAX_EPOCHS+1):
    avg_cost = []
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE,
                        shuffle=True, drop_last=False)
    for x, y in (dataloader):
      cost = update_weights(model, x, y)

      avg_cost.append(cost)
    # Display logs per each DISPLAY_STEP
    if (epoch) % DISPLAY_STEP == 0:
      print ("Epoch: {:04d} cost={:.9f}".format(epoch, np.mean(avg_cost)))
train(model, train_dataset)
print ("Optimization Finished!")

In [ ]:
print("Training datset", evaluate(model, train_dataset))
print("Test datset", evaluate(model, test_dataset))